In [ ]:
!nvidia-smi

Sun Nov 17 11:28:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -q inference supervision

In [ ]:
!git clone https://github.com/SkalskiP/yolov9.git
%cd yolov9
!pip install -r requirements.txt -q
!pip install -q roboflow

fatal: destination path 'yolov9' already exists and is not an empty directory.
/content/yolov9


In [ ]:
pip install inference


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 9.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 839.6/839.6 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.9/891.9 kB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from IPython.display import Image
import cv2
import random
import getpass
from tqdm import tqdm
import supervision as sv
from roboflow import Roboflow
from inference import get_model
%matplotlib inline
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
%cd {HOME}/yolov9


rf = Roboflow(api_key="XaYRaTIxrFcAN0CfHjEN")
project = rf.workspace("ahmed-ws").project("ball-detection-3afaq")
version = project.version(5)
dataset = version.download("yolov9")


/content/yolov9
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ball-detection-5 in yolov9:: 100%|██████████| 686/686 [00:00<00:00, 8291.77it/s]


In [ ]:
model = get_model(model_id="ball-detection-3afaq/5", api_key="XaYRaTIxrFcAN0CfHjEN")

Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
Specified provider 'OpenVINOExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
Specified provider 'CoreMLExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'


In [ ]:
dataset.location

'/content/yolov9/ball-detection-5'

In [ ]:
import cv2
import os
from PIL import Image
from IPython.display import display

def detect_ball(image):
    result = model.infer(image, confidence=0.3)[0]
    detections = sv.Detections.from_inference(result)

    label_annotator = sv.LabelAnnotator(text_color=sv.Color.BLACK)
    bounding_box_annotator = sv.BoundingBoxAnnotator()

    annotated_image = image.copy()
    annotated_image = bounding_box_annotator.annotate(scene=annotated_image, detections=detections)
    annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

    pil_image = Image.fromarray(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))

    return pil_image

In [ ]:
from tqdm import tqdm

def process_video(filepath):
    output_folder = 'runtime_processed_images'

    if os.path.exists(output_folder) and os.path.isdir(output_folder):
        for filename in os.listdir(output_folder):
            file_path = os.path.join(output_folder, filename)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
            except Exception as e:
                print(f"Error: {e}")

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(filepath)
    if not cap.isOpened():
        print("Error: Could not open video file.")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    interval = int(fps / 8)

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frame_count = 0
    c = 0
    with tqdm(total=total_frames, desc="Processing Video...") as progress_bar:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            if frame_count % interval == 0:
                annotated_frame = detect_ball(frame)
                filename = os.path.join(output_folder, f"{c}_frame.jpg")
                c += 1
                annotated_frame.save(filename)

            frame_count += 1
            progress_bar.update(1)

    cap.release()

In [ ]:
fp = "/content/lbw-dataset/v20.mp4"
process_video(fp)

Processing Video...: 100%|██████████| 111/111 [00:49<00:00,  2.24it/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:


# Path to your image sequence folder
path = '/content/yolov9/runtime_processed_images'

# Output folder where the video will be saved
out_path = '/content/yolov9'

# Output video name
out_video_name = 'output_video.mp4'

# Full path of the output video
out_video_full_path = os.path.join(out_path, out_video_name)

# Get the list of image filenames in the folder
pre_imgs = os.listdir(path)

# Sort the image filenames numerically based on the frame numbers
pre_imgs_sorted = sorted(pre_imgs, key=lambda x: int(x.split('_')[0]))

# Initialize an empty list to store the full paths of the images
img = []

# Construct the full paths of the images
for img_name in pre_imgs_sorted:
    img.append(os.path.join(path, img_name))

# Initialize VideoWriter parameters
cv2_fourcc = cv2.VideoWriter_fourcc(*'mp4v')
frame = cv2.imread(img[0])
size = list(frame.shape)
del size[2]
size.reverse()

# Initialize VideoWriter object
video = cv2.VideoWriter(out_video_full_path, cv2_fourcc, 10, size) #output video name, fourcc, fps, size

# Iterate over each image and write it to the video
for i in range(len(img)):
    video.write(cv2.imread(img[i]))

# Release the VideoWriter object
video.release()

print('Video Sucessfully Processed!!!')


NameError: name 'os' is not defined